In [1]:
import rasterio
import numpy as np
from rasterio import windows
from itertools import product

# Window loading

In [2]:
def get_windows(src, overlap, width, height, boundless=True):
    offsets = product(range(0, src.meta['width'], width), range(0, src.meta['height'], height))
    full_window = windows.Window(col_off=0, row_off=0, width=src.meta['width'], height=src.meta['height'])
    for col_off, row_off in offsets:
        window = windows.Window(
            col_off=col_off - overlap,
            row_off=row_off - overlap,
            width=width,
            height=height)
        
        if boundless:
            yield window
        else:
            yield window.intersection(full_window)


In [3]:
width = 20
height = 20
overlap = 5

source_data = "../../data/TestDataSet/S2_comp_first.tif"
outdir = "./retiled_images/"

from pathlib import Path
Path(outdir).mkdir(parents=True, exist_ok=True)


In [4]:
%%time
import xarray as xr
with rasterio.open(source_data) as src:
    window_list = [w for w in get_windows(src, overlap, width, height)]
    arrays = [src.read([1, 2, 3], window=window, boundless=False) for window in window_list]

CPU times: user 16.4 s, sys: 659 ms, total: 17 s
Wall time: 20.2 s


In [5]:
outpath_list = [outdir+'tile_{}.tif'.format(count) for count in range(len(arrays))]

In [6]:
%%time
import concurrent.futures

with rasterio.open(source_data) as src:
    def write_array(array, window, outpath):
        profile = src.profile
        profile.update(
            transform=src.window_transform(window),
            height=window.height,
            width=window.width)
        with rasterio.open(outpath, 'w', **profile) as dst:
            dst.write(array)
    
    with concurrent.futures.ThreadPoolExecutor(
            max_workers=8
        ) as executor:
            futures = executor.map(write_array, arrays, window_list, outpath_list)

CPU times: user 31min 56s, sys: 8min 53s, total: 40min 50s
Wall time: 9h 21min 10s
